# Generate morphotope geometry with cluster information

Dissolves enclosed tessellation cells into morphotopes for visualisation purposes and links cluster information we have on buildings to each morphotope. To be displayed on the interactive map.

In [1]:
import os

import geopandas as gpd
import pandas as pd

from libpysal import graph
from tqdm.auto import tqdm

In [2]:
folder = "/data/uscuni-ulce/processed_data/tessellations/"
clusters_folder = "/data/uscuni-ulce/processed_data/clusters/"
morphotope_folder = "/data/uscuni-ulce/processed_data/morphotope_clusters/"

In [3]:
# os.mkdir(morphotope_folder)

In [4]:
regions = gpd.read_parquet("/data/uscuni-ulce/regions/cadastre_regions_hull.parquet")

In [5]:
def generate_morphotope_geometry(region_id):
    tess = gpd.read_parquet(f"{folder}tessellation_{region_id}.parquet")
    blg = gpd.read_parquet(f"{clusters_folder}{region_id}_clusters.pq")
    tess_w_data = tess.join(blg[["morph", "final", "regional", "final_without_noise"]], how="inner")

    morphotopes = tess_w_data[~tess_w_data["morph"].str.contains("-1")].dissolve("morph")
    morphotopes.geometry = morphotopes.buffer(.01)

    noise = tess_w_data[tess_w_data["morph"].str.contains("-1")]
    contig = graph.Graph.build_fuzzy_contiguity(noise, buffer=0.1)
    noise_morphotopes = noise.dissolve(contig.component_labels)
    noise_morphotopes.geometry = noise_morphotopes.buffer(.01)
    complete_morphotopes = pd.concat([morphotopes, noise_morphotopes.set_index("morph")])

    complete_morphotopes.to_parquet(f"{morphotope_folder}{region_id}_clusters.pq")

In [6]:
for region in tqdm(regions.index[::-1]):
    generate_morphotope_geometry(region)

  0%|          | 0/847 [00:00<?, ?it/s]